In [2]:
!pip install pymorphy2

     |████████████████████████████████| 51kB 1.7MB/s 
     |████████████████████████████████| 7.1MB 6.3MB/s 


In [18]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
import requests
from bs4 import BeautifulSoup
import urllib
import pickle
import os

import google.oauth2.credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

import googleapiclient.discovery

from nltk import word_tokenize
from nltk.corpus import stopwords

import pymorphy2

import pandas as pd


from gensim.models import FastText

In [0]:
CLIENT_SECRETS_FILE = "client_secret.json"
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'
os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'

In [0]:
def get_authenticated_service():
    credentials = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            credentials = pickle.load(token)
    #  Check if the credentials are invalid or do not exist
    if not credentials or not credentials.valid:
        # Check if the credentials have expired
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                CLIENT_SECRETS_FILE, SCOPES)
            credentials = flow.run_console()
 
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(credentials, token)
 
    return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)
 

In [9]:
service = get_authenticated_service()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=499464782258-sk1tjt846grh4bartqo42538n81bkb8o.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=Oca5D3MIJ8Zis7bEcswymKY9BS14d5&prompt=consent&access_type=offline
Enter the authorization code: 4/zAGtlvwC0AAI1Fe4hbNYb3jGeQVKxew5tr1Up5cpdXBzyKs3FLLjyNU


In [0]:
def get_video_comments(service, **kwargs):
    summary = []
    results = service.commentThreads().list(**kwargs).execute()
    
    while results:
        for item in results['items']:
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            num_like = item['snippet']['topLevelComment']['snippet']['likeCount']
            num_reply = item['snippet']['totalReplyCount']
            summary.append({
                'comment' : comment,
                'num_like' : num_like,
                'num_reply' : num_reply
            })
 
        if 'nextPageToken' in results:
            kwargs['pageToken'] = results['nextPageToken']
            results = service.commentThreads().list(**kwargs).execute()
        else:
            break
 
    return pd.DataFrame(summary)

In [0]:
def check_word(word):
    if word.isalpha() and word not in stop_words:
        return morph.parse(word)[0].normal_form

    
def tokenize_sentence(sentence):
    words = word_tokenize(sentence)
    norm_words = []
    for word in words:
        cur_word = check_word(word)
        if cur_word is not None:
            norm_words.append(cur_word)
    return norm_words


def add_tokenize_columns(df, tokenized_col='tokenize_text', text_for_tokenize='comment', language='russian'):
    global stop_words, morph
    stop_words = set(stopwords.words(language))
    morph = pymorphy2.MorphAnalyzer()
    df[tokenized_col] = df[text_for_tokenize].apply(tokenize_sentence)
    return df

In [0]:
summary = get_video_comments(service, part='snippet', videoId='kOvdx1sz--U', textFormat='plainText')

In [32]:
%%timeit
add_tokenize_columns(summary)

1 loop, best of 3: 3 s per loop


In [0]:
summary = add_tokenize_columns(summary)

In [31]:
summary

,comment,num_like,num_reply,tokenize_text
0,"Почему пивнухи, горилки , кб работают, а на ту...",0,0,"[почему, пивнуха, горилка, кб, работать, турник]"
1,Ой как не культурно !!!!!\nНо пиздец как прави...,0,0,"[ой, культурно, но, пиздец, правильно, ебаный,..."
2,Настоящий мужик,0,0,"[настоящий, мужик]"
3,Суппер спортсмен правельно всех на х...й посла...,0,0,"[суппереть, спортсмен, правельный, х, й, посла..."
4,Журналисты продажные🤦‍♂️👍,0,0,[журналист]
...,...,...,...,...
2199,Журналюги клоуны и жополизы редкостные,1160,8,"[журналюга, клоун, жополиза, редкостный]"
2200,Наконец то быдло мэру ответили в его манере! ),1676,9,"[наконец, быдло, мэр, ответить, манера]"
2201,А в СМИ то не так показывают. Спортсмен то кра...,86,4,"[а, сми, показывать, спортсмен, красавчик]"
2202,"Спортплошадка запрещена, конопля запрещена. Но...",439,29,"[спортплошадка, запретить, конопля, запретить,..."


In [0]:
model = FastText(size=4, window=3, min_count=1)

In [0]:
model.build_vocab(summary['tokenize_text'])

In [0]:
model.train(sentences=summary['tokenize_text'], total_examples=len(summary['tokenize_text']), epochs=10) 